In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
from torch.autograd import Variable
from sklearn.decomposition import PCA
import pandas as pd
import copy
import os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
import matplotlib
from matplotlib.pyplot import figure

In [ ]:
def load_data(train_batch_size,test_batch_size):
    # Fetch training data: total 60000 samples
    train_loader = torch.utils.data.DataLoader(
        torchvision.datasets.MNIST('data', train=True, download=True,
                       transform=transforms.Compose([ transforms.ToTensor()  ])),
        batch_size=train_batch_size, shuffle=True)

    # Fetch test data: total 10000 samples
    test_loader = torch.utils.data.DataLoader(
        torchvision.datasets.MNIST('data', train=False, transform=transforms.Compose([transforms.ToTensor() ])),
        batch_size=test_batch_size, shuffle=True)

    return (train_loader,test_loader)
train_batch_size = 1000
test_batch_size = 1000
train_loader, test_loader = load_data(train_batch_size, test_batch_size)

In [ ]:
class Model(nn.Module):
    def __init__(self,):
        super().__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128,10)

    def forward(self, x):

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
loss_function = nn.CrossEntropyLoss()

In [ ]:
def Calculate_Losses(model, loader):
    correct_class = 0
    totalVal = 0
    costTotal = 0
    costCounter = 0
    with torch.no_grad():
        for batch in loader:
            img, target = batch
            output = model(img.view(-1, 784))
            cost = loss_function(output, target)
            costTotal += cost
            costCounter += 1
            for i, outputTensor in enumerate(output):
                if torch.argmax(outputTensor) == target[i]:
                    correct_class += 1
                totalVal += 1
    return costTotal / costCounter, round(correct_class/totalVal, 3)

In [ ]:
train_loss1 = []
test_loss1 = []
train_acc1 = []
test_acc1 = []

In [ ]:
def train(model, number_epochs):

    model.train()
    epoch = 0
    dataf = pd.DataFrame()

    for epoch in range (number_epochs):
        epoch += 1

        for _, (imgs, targets) in enumerate(train_loader):
            imgs, targets = Variable(imgs),Variable(targets)
            optimizer.zero_grad()
            prediction = model(imgs.view(-1, 784))
            loss = loss_function(prediction, targets)
            loss.backward()
            optimizer.step()
        temp_dataf = pd.DataFrame()
        #if(epoch%3==0):
            
        for name, parameter in model.named_parameters():
                if 'weight' in name:
                    weights = torch.nn.utils.parameters_to_vector(parameter).detach().numpy()
                    temp_dataf = pd.concat([temp_dataf, pd.DataFrame(weights).T], axis = 1)
        dataf = pd.concat([dataf, temp_dataf], axis = 0)
        train_loss, train_acc = Calculate_Losses(Model_1, train_loader)
        test_loss, test_acc = Calculate_Losses(Model_1, test_loader)

        train_loss1.append(train_loss)
        test_loss1.append(test_loss)
        train_acc1.append(train_acc)
        test_acc1.append(test_acc)

    return dataf

In [ ]:
max_epochs = 45
all_dataf = pd.DataFrame()
columns=["x","y","Repetition"]

#loss_fn = nn.CrossEntropyLoss()
for count in range(8):
    print("Repetition: "+str(count))
    Model_1 = Model()
    optimizer = torch.optim.Adam(Model_1.parameters(),lr = 0.001, weight_decay=1e-4)
    model_name1 = "Repetition: "+str(count)
    temp_dataf = train(Model_1, max_epochs)
    
    all_dataf = pd.concat([all_dataf,temp_dataf],ignore_index=True)

In [ ]:
dataf=all_dataf

In [ ]:
dataf=np.array(dataf)

In [ ]:
pca=PCA(n_components=2)

In [ ]:
smaller_data=pca.fit_transform(dataf)

In [ ]:
print(f"Length of train_acc1: {len(train_acc1)}")
print(f"Length of dataf: {len(dataf)}")
print(f"Length of train_acc1: {len(train_loss1)}")


In [ ]:
dataf = pd.DataFrame(smaller_data, columns=['x','y'])
dataf['Accuracy'] = train_acc1
temp_dataf['Accuracy'] = train_acc1[:len(temp_dataf)]
dataf['Loss'] = train_loss1
final_dataf = dataf.iloc[::3, :]

In [ ]:
for i in range(120):
    m = list(final_dataf['Accuracy'])[i]
    plt.scatter(final_dataf['x'][i*3], final_dataf['y'][i*3], marker = f'${m}$')
    plt.title("PCA for model")

In [ ]:
level1 = all_dataf.iloc[ : , 0:7840]
dataf = level1
dataf = np.array(df)
pca = PCA(n_components=2)
smaller_data = pca.fit_transform(dataf)

In [ ]:
for i in range(120):
    m = list(final_dataf['Accuracy'])[i]
    plt.scatter(final_dataf['x'][i*3], final_dataf['y'][i*3], marker = f'${m}$')
    plt.title("PCA for Layer 1")